In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

### Feature Selection Analysis wiht Sentiment for EIHOTEL.BO

In [5]:
df = pd.read_csv('../../datasets/processed_data/combined_features/EIHOTEL.BO.csv')

In [6]:
df.columns

Index(['date', 'open', 'close', 'adj close', 'volume', 'low', 'high',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'inr=x_percent_change', 'cl=f_percent_change',
       'treasury_yeild_10_years_percent_change', 'usdx-index_percent_change',
       '^nsei_percent_change', '^bsesn_percent_change', '^gspc_percent_change',
       'hsi_percent_change', 'sha_percent_change', '^sti_percent_change'],
      dtype='object', length=635)

In [7]:
def create_custom_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates the custom target price, which is computed as ln(high/yesterday_close)
    """    
    # make a copy of the dataframe so as not to change the original dataframe
    data_df = df.copy()
    
    # create yesterday_close feature as
    data_df['yesterday_close'] = data_df['close'].shift(1)
    
    # create custom target price to predict, computing  ln(high/yesterday_close)
    data_df['ln_target'] = np.log(data_df['high'] / data_df['yesterday_close'])
    
    # as yesterday close would not be available for first day, 
    # we would not have custom target price for that day, which needs to be excluded 
    return(data_df.iloc[1:, ])

In [8]:
df = create_custom_target(df)

In [9]:
# Drop cols with NaN values
df = df.dropna(axis=1, how='all')
df.fillna(method='ffill', inplace=True)

In [10]:
combined_date_df = df['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)

In [11]:
df_without_date = df.drop(columns=["date"])

In [12]:
df_without_date.dropna(inplace=True)

In [13]:
def timeseries_to_supervise(df, window_size, target):    
    X = []
    y = []
    indx = []
    no_records = len(df)
    #     
    for i in range(window_size, no_records):
        X.append(df.iloc[i-window_size:i].drop(target, axis=1).values.flatten())  # Collect past records as a sequence
        y.append(df.iloc[i][target])  # Next record as target variable
        indx.append(np.arange(i-window_size, i))

    X = pd.DataFrame(X)
    y = pd.Series(y)
    return(X, y, indx)

In [14]:
window_size = 10

In [15]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# Identify the indices of the specified sentiment columns in the original dataset
sentiment_columns = ['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound']
column_indices = [train_data.columns.get_loc(col) for col in sentiment_columns if col in train_data.columns]

column_indices

[616, 618, 619, 621, 623, 622]

In [16]:
# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [17]:
# Use SelectKBest to select the top 44 features
selector = SelectKBest(score_func=f_regression, k=44)
X_train_44 = selector.fit_transform(X_train, y_train)
X_test_44 = selector.transform(X_test)

# Determine if the specified columns are among the top 44 features
selected_indices = selector.get_support(indices=True)
missing_indices = [idx for idx in column_indices if idx not in selected_indices]

# Add the specified columns if they are not among the top 44 features
for idx in missing_indices:
    extracted_column_train = X_train.iloc[:, idx].values.reshape(-1, 1)
    extracted_column_test = X_test.iloc[:, idx].values.reshape(-1, 1)
    X_train_44 = np.hstack([X_train_44, extracted_column_train])
    X_test_44 = np.hstack([X_test_44, extracted_column_test])

# If some of the specified columns were already in the top 44, 
# select additional top features to make the total count 50
remaining_indices = [i for i in range(X_train.shape[1]) if i not in selected_indices and i not in missing_indices]
num_additional_features_needed = 50 - X_train_44.shape[1]

if num_additional_features_needed > 0:
    additional_selector = SelectKBest(score_func=f_regression, k=num_additional_features_needed)
    additional_selector.fit(X_train.iloc[:, remaining_indices], y_train)
    X_train_additional = additional_selector.transform(X_train.iloc[:, remaining_indices])
    X_test_additional = additional_selector.transform(X_test.iloc[:, remaining_indices])

    X_train_44 = np.hstack([X_train_44, X_train_additional])
    X_test_44 = np.hstack([X_test_44, X_test_additional])

X_train_50 = X_train_44
X_test_50 = X_test_44

In [18]:
linear_reg = LinearRegression()

param_grid = {
    'fit_intercept': [True, False],         
}

grid_search = GridSearchCV(linear_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_50, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

model_50 = grid_search.best_estimator_
model_50.fit(X_train_50, y_train)

Best parameters:  {'fit_intercept': True}
Best cross-validation score:  0.0005693704145646624


LinearRegression()

In [19]:
def convert_custom_target_to_actual_for_supervise(df: pd.DataFrame, window: int, y: "pd.Series[int]") -> "pd.Series[int]":
    """
    this module converts custom target - ln(high/yesterday_close) to actual high price again for timeseries converted data using rolling window of size 10
    """
    data_df = df.copy()
    
    # exclude first 10 rows of train/test data, as while us
    
    y = np.exp(y) * data_df.loc[data_df.index[window:], 'yesterday_close'].reset_index(drop=True)
    return(y)    

In [20]:
def evaluate_model(model, window, test_data, test_date, X_test, y_test):
    
    # do target prediction using the provide model
    y_pred = model.predict(X_test)

    # convert back to original value, before computing mape            
    y_test = convert_custom_target_to_actual_for_supervise(test_data, window, y_test)
    y_pred = convert_custom_target_to_actual_for_supervise(test_data, window, y_pred)

    test_dates = test_date[window:].reset_index(drop=True)
    predictions_df = pd.DataFrame({'date': test_dates, 'y_test': y_test, 'y_pred': y_pred})

    # compute regression metric - mape 
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # compute rmse metric
    rmse = mean_squared_error(y_test, y_pred, squared=False)        
    return(predictions_df, mape, rmse)

In [21]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [22]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2020-06-23   74.900002   71.789440
 1    2020-06-24   74.949997   75.545094
 2    2020-06-25   68.900002   70.614820
 3    2020-06-26   69.000000   68.462157
 4    2020-06-29   68.750000   67.492877
 ..          ...         ...         ...
 722  2023-05-24  205.600006  203.920435
 723  2023-05-25  215.850006  204.006124
 724  2023-05-26  207.699997  208.412543
 725  2023-05-30  207.850006  208.066956
 726  2023-05-31  210.750000  210.293323
 
 [727 rows x 3 columns],
 0.014740049805841737,
 3.225891342518545)

In [23]:
print(f"MAPE for model with top 50 features: {mape*100:.2f}%")

MAPE for model with top 50 features: 1.47%


In [24]:
predictions_df.to_csv("lr_EIHOTEL.csv", index=False)

In [25]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df_copy = predictions_df.copy(deep=True)


predictions_df_copy['date'] = pd.to_datetime(predictions_df_copy['date'])

predictions_df_copy['label'] = 'Actual'
predictions_df_copy['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df_copy).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df_copy).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)

In [29]:
feature_names = df.columns.tolist()

# Get coefficients from the model
coefficients = model_50.coef_

# Map coefficients to corresponding feature names
feature_coefficient_mapping = dict(zip(feature_names, coefficients))

# Sort by absolute coefficient values (magnitude)
sorted_features = sorted(feature_coefficient_mapping.items(), key=lambda x: abs(x[1]), reverse=True)

# Display top 10
for feature, coef in sorted_features[:10]:
    print(f"{feature}: {coef}")

trend_trix: -0.04321071232458663
trend_ema_slow: -0.039510922214719954
trend_vortex_ind_pos: 0.009715991298718803
trend_macd: 0.008356625565513207
trend_mass_index: 0.006006315204742178
volatility_dch: -0.005760823758252802
trend_vortex_ind_neg: -0.003895383597554771
volatility_kcc: -0.002242658524046953
volatility_bbhi: 0.0019172872699602837
volatility_dcp: 0.0019009608496743774


### Feature Selection Analysis wihtout Sentiment for EIHOTEL.BO

In [30]:
df = pd.read_csv('../../datasets/processed_data/combined_features/EIHOTEL.BO.csv')

In [31]:
df_without_sentiment = df.drop(columns=['agg_polarity', 'agg_compound', 'topic_polarity', 'topic_compound', 'ticker_polarity', 'ticker_compound'])

In [32]:
df_without_sentiment = create_custom_target(df_without_sentiment)

In [33]:
# Drop cols with NaN values
df_without_sentiment = df_without_sentiment.dropna(axis=1, how='all')
df_without_sentiment.fillna(method='ffill', inplace=True)

In [34]:
combined_date_df = df_without_sentiment['date']
train_date, test_date = train_test_split(combined_date_df, train_size=0.8, shuffle=False)
dev_date, test_date = train_test_split(test_date, train_size=0.5, shuffle=False)

In [35]:
df_without_date = df_without_sentiment.drop(columns=["date"])

In [36]:
df_without_date.dropna(inplace=True)

In [37]:
# do train/test split the data with shuffle = False
train_data, test_data = train_test_split(df_without_date, train_size=0.8, shuffle=False)

# convert timeseries to be used in supervise learning model
X_train, y_train, indx_train = timeseries_to_supervise(train_data, window_size, 'ln_target')  

# convert timeseries to be used in supervise learning model    
X_test, y_test, indx_test = timeseries_to_supervise(test_data, window_size, 'ln_target')  

In [38]:
selector = SelectKBest(score_func=f_regression, k=50)
X_train_50 = selector.fit_transform(X_train, y_train)
X_test_50 = selector.transform(X_test)

In [39]:
linear_reg = LinearRegression()

param_grid = {
    'fit_intercept': [True, False],         
}

grid_search = GridSearchCV(linear_reg, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_50, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

model_50 = grid_search.best_estimator_
model_50.fit(X_train_50, y_train)

Best parameters:  {'fit_intercept': True}
Best cross-validation score:  0.0005592916267689013


LinearRegression()

In [40]:
predictions_df, mape, rmse = evaluate_model(model_50, window_size, test_data, test_date, X_test_50, y_test)

In [41]:
predictions_df, mape, rmse

(           date      y_test      y_pred
 0    2021-12-14   74.900002   71.720636
 1    2021-12-15   74.949997   75.475615
 2    2021-12-16   68.900002   70.531469
 3    2021-12-17   69.000000   68.378703
 4    2021-12-20   68.750000   67.408828
 ..          ...         ...         ...
 722         NaN  205.600006  204.025396
 723         NaN  215.850006  203.870157
 724         NaN  207.699997  208.377461
 725         NaN  207.850006  208.067773
 726         NaN  210.750000  210.268789
 
 [727 rows x 3 columns],
 0.01475590264113978,
 3.2345235345158136)

In [42]:
predictions_df.to_csv("lr_EIHOTEL_without_senitment.csv", index=False)

In [43]:
print(f"MAPE for model with top 50 features without sentiment features: {mape*100:.2f}%")

MAPE for model with top 50 features without sentiment features: 1.48%


In [44]:
feature_names = df.columns.tolist()

# Get coefficients from the model
coefficients = model_50.coef_

# Map coefficients to corresponding feature names
feature_coefficient_mapping = dict(zip(feature_names, coefficients))

# Sort by absolute coefficient values (magnitude)
sorted_features = sorted(feature_coefficient_mapping.items(), key=lambda x: abs(x[1]), reverse=True)

# Display top 10
for feature, coef in sorted_features[:10]:
    print(f"{feature}: {coef}")

trend_ema_slow: 0.008691029667158612
trend_macd: -0.005667460910816607
volatility_kcli: -0.002336646354381545
volatility_kcp: 0.0019967036164571755
volatility_bbw: -0.001949270189513415
trend_sma_fast: 0.001838875863834
volatility_atr: -0.0017919818246192112
trend_sma_slow: -0.0016153659050462022
open: -0.0014246710452106204
volatility_ui: 0.0013754877976213567


In [46]:
import altair as alt

alt.themes.enable('fivethirtyeight')

predictions_df['date'] = pd.to_datetime(predictions_df['date'])

predictions_df['label'] = 'Actual'
predictions_df['predicted_label'] = 'Predicted'

# Actual high price line
line1 = alt.Chart(predictions_df).mark_line(strokeWidth=2).encode(
    x='date:T',
    y=alt.Y('y_test:Q', title='Price', scale=alt.Scale(zero=False)),
    color=alt.Color('label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Predicted high price line
line2 = alt.Chart(predictions_df).mark_line(strokeWidth=1, strokeDash=[3, 3]).encode(
    x='date:T',
    y=alt.Y('y_pred:Q', title='', scale=alt.Scale(zero=False)),
    color=alt.Color('predicted_label:N', legend=alt.Legend(title="Line Type")),
    tooltip=['date', 'y_test', 'y_pred']
)

# Combine the two lines
chart = alt.layer(line1, line2).properties(
    title='Actual vs Predicted High Prices',
    width=650,
    height=400
).interactive()

chart

alt.LayerChart(...)